# Lesson: Securing the Universe of User Data: Unveiling Password Security, Hashing, Encryption, and bcrypt Implementation

### Introduction to Securing User Data

Hello, programming friends! Are you prepared to delve deep into the realm of secure user data? This particular lesson underscores the importance of password security in safeguarding user data. It covers essential concepts including hashing, encryption, and the use of bcrypt for password encryption. Buckle up! By the end of our exciting journey, you will have mastered the use of bcrypt for password security.

Imagine this scenario. You are storing user data, such as a username and a password, in your database. Suddenly, an uninvited guest (an attacker) gains access to this data. These pieces of information, as innocuous as they may seem, can provide an attacker with significant insights about their target, leading to misuse or unauthorized access. So, how can we prevent this? By securing user data, of course!

### User Data and Its Importance. Hashing vs. Encryption

User data often includes unique identifiers like a username or email and a password for authentication. This data is critical - if it falls into the wrong hands, someone could commit unauthorized activities. It's like a library card system: if an intruder gains access to a user's library card number and password, they could misuse the information, leading to unauthorized checkouts.

Passwords are central to securing user accounts. Weak passwords like '12345' or 'password' offer easy access to attackers. Hence, robust passwords are endorsed, typically at least 12 characters mixed with uppercase, lowercase, numbers, and symbols. For instance, 'My$3cur3Pa$$w0rd!' offers better security than '12345'.

To store passwords securely, we employ hashing and encryption rather than plain texts. Encryption changes plain text into cipher text using an encryption key, which can be decrypted using a decryption key. In contrast, hashing is irreversible. It processes input into a fixed byte size, like blending fruits for a smoothie where retrieving the original pieces is impossible. Hashing obeys the same rule.

### Introduction to bcrypt

Bcrypt, a password-hashing function created by Niels Provos and David Mazières, adds an extra layer of security to our data. Bcrypt applies both salt and hash techniques on passwords, enhancing their security.

In JavaScript, bcrypt uses an asynchronous API with native promises to hash passwords. We hash a password using the bcrypt.hash() function and check a password against a hash using the bcrypt.compare() function.

Here's a direct example of using the bcrypt.hash() function:

In [ ]:
const password = 'My$3cur3Pa$$w0rd!';
const saltRounds = 10;

try {
  bcrypt.hash(password, saltRounds, function(err, hash) {
    if (err) {
      throw new Error('Hashing Failed');
    }
    console.log(hash);
  });
} catch (err) {
  console.error(err.message);
}

Let's dwell a little deeper into the bcrypt.hash() function. It accepts three parameters:

    password: This is the plain-text password that you want to hash.

    saltRounds: This is the number of rounds you want to process the data for. More rounds lead to more secured hashed data but require more processing time. A balance between security and performance is usually maintained at around ten rounds.

    callback: This function gets executed once bcrypt has hashed the password. It should accept two arguments:
        error: An error object if an error occurred, null otherwise.
        hash: If no error occurred, this is the hashed password.

### Implementing bcrypt for Password Security

We move now to the verification part with bcrypt.compare(). Here, errors can be handled by examining the err parameter in the callback function. If err is not null, an error has occurred during the hash comparison.

In [ ]:
const passwordAttempt = 'TryingToGuessPassword';

bcrypt.compare(passwordAttempt, hash, function(err, isMatch) {
    if (err) {
      throw new Error('Comparison failed!');
    } else if (!isMatch) {
      console.log("Password doesn't match!");
    } else {
      console.log("Password matches!");
    }
});

In this script, passwordAttempt refers to the password we want to check, and hash is the user's hashed password from the database. isMatch will be either true or false, indicating whether the password and hash match.

### Beefing Up Security with Middleware: Hashing Passwords in Signup and Login Routes

Middlewares refer to functions that have access to the request object, the response object, and the next middleware function in the application's request-response cycle. They can execute any code, make changes to the request-response objects, end the request-response cycle, and call the next middleware function in the stack.

With the help of middlewares, we can add an extra layer of security to our application by hashing the passwords when users sign up or log in. Let's explore how!

First, we'll define a middleware function for password hashing:

In [ ]:
function hashPassword(req, res, next){
  const saltRounds = 10;

  // hash the password
  bcrypt.hash(req.body.password, saltRounds, function(err, hash) {
    if (err) {
      console.log(err);
      res.status(500).send();
    } else {
      // store the hashed password to the request body
      req.body.password = hash;
      // move to the next middleware function/route handler
      next();
    }
  });
}

This hashPassword middleware hashes the password from the request body and replaces the original password in the request body with the hashed password. If an error occurs during hashing, it will send a 500 status code.

Now, we will create the signup and login routes. We'll be storing the user data in a hardcoded object for simplicity. In your actual applications, this data will be stored in a database instead.

In [ ]:
// Hardcoded user data
let users = {};

// Signup route
app.post('/signup', hashPassword, function(req, res) {
  const { username, password } = req.body;
  
  // Check if the username is already taken
  if (users[username]) {
    res.status(409).send('Username is already taken!');
  } else {
    // Save the user data
    users[username] = { password };
    res.status(200).send('User created successfully!');
  }
});

// Login Route
app.post('/login', function(req, res) {
  const { username, password } = req.body;
  
  // Find the user in our hardcoded data
  let user = users[username];
  
  if (user) {
    // Compare the hash and respond
    bcrypt.compare(password, user.password, function(err, match) {
      if (err) {
        res.status(500).send();
      } else if (match) {
        res.status(200).send('Login successful!');
      } else {
        res.status(401).send('Password is incorrect!');
      }
    });
  } else {
    res.status(404).send('User not found!');
  }
});

The signup route uses the hashPassword middleware to hash the password before saving the user data. The login route checks the username and password, compares the hashed password with the entered password, and sends the appropriate response.

And voila! You now know how to use middlewares in Express.js to provide an extra layer of security by hashing passwords! Remember, bcrypt comes pre-installed in most CodeSignal environments. However, learning to do it is crucial for working on your own projects or other environments.

### Lesson Summary and Practice Announcement

Bravo! You've navigated the sea of user data, grasped the importance of password security, unearthed the secrets of hashing and encryption, and wielded bcrypt for password security.

Up next, we will use the tools we've acquired to build something tangible in the form of practice sessions. These exercises will test your understanding and help you tackle real-world data security problems. Let's roll up our sleeves and dive into some coding!


In [ ]:
// index.js (Express server)
const express = require('express');
const bcrypt = require('bcrypt');
const app = express();
app.use(express.json());
const port = 5000;

app.post('/api/signup', async (req, res) => {
    const { password } = req.body;
    const saltRounds = 10;
    try {
        const hashedPassword = await bcrypt.hash(password, saltRounds);
        // Save hashedPassword to database (mocked)
        console.log('Password hashed and saved: ', hashedPassword);
        res.status(201).send({ hashedPassword });
    } catch (error) {
        res.status(500).send('Error during signup.');
    }
});

app.listen(port, () => console.log(`Server running on port ${port}`));

In our Library System Management scenario, we need to secure users' passwords during the signup process. The code provided for the server-side uses bcrypt to hash a password, whereas the client-side features a simple signup interface with password input. Click Run to find out how it works!

In [ ]:
const password = 'My$3cur3Pa$$w0rd!';
const saltRounds = 10;

try {
  bcrypt.hash(password, saltRounds, function(err, hash) {
    if (err) {
      throw new Error('Hashing Failed');
    }
    console.log(hash);
  });
} catch (err) {
  console.error(err.message);
}

In [ ]:
// App.js
import { useState } from 'react';
import axios from 'axios';

export default function App() {
  const [password, setPassword] = useState('');
  const [hashedPassword, setHashedPassword] = useState('');

  const handleSignUp = () => {
    axios.post('/api/signup', { password }).then(response => {
      setHashedPassword(response.data.hashedPassword);
    });
  };

  return (
    <div>
      <input
        type="password"
        placeholder="Enter your password"
        value={password}
        onChange={(e) => setPassword(e.target.value)}
      />
      <button onClick={handleSignUp}>Sign Up</button>
      {hashedPassword && (
        <div>
          <p>Signed up with hashed password: {hashedPassword}</p>
        </div>
      )}
    </div>
  );
}

Good going, Galactic Pioneer! It's time to strengthen our password security by tweaking a parameter. In the provided server code, adjust the number of hashing rounds (saltRounds) used by bcrypt to make the storage of passwords even more secure.


In [ ]:
const bcrypt = require('bcrypt');
const app = require('express')();
const port = 5000;

app.use(require('express').json());

app.use('/api/signup', (req, res, next) => {
  const { password } = req.body;
  if (!password) {
    return res.status(400).send('Password is required');
  }
  
  bcrypt.hash(password, 12, (err, hash) => { // TODO: Tweak the security level here by setting 'saltRounds' parameter value to 12
    if (err) {
      return res.status(500).send('Error hashing password');
    }
    req.body.passwordHash = hash;
    next();
  });
});

app.post('/api/signup', (req, res) => {
  const { username, passwordHash } = req.body;
  res.status(200).send(`User ${username} signed up successfully!`);
});

app.listen(port, () => {
  console.log(`Server is listening at http://localhost:${port}`);
});

In [ ]:
//App.js (React frontend)
import { useState } from 'react';
import axios from 'axios';

const App = () => {
  const [username, setUsername] = useState('');
  const [password, setPassword] = useState('');
  const [message, setMessage] = useState('');

  const handleSignup = () => {
    axios.post('/api/signup', { username, password })
      .then(response => {
        setMessage(response.data);
      })
      .catch(error => {
        setMessage('Signup failed. Please try again.');
      });
  };

  return (
    <div>
      <input type="text" value={username} onChange={e => setUsername(e.target.value)} placeholder="Library Card Number" />
      <input type="password" value={password} onChange={e => setPassword(e.target.value)} placeholder="Password" />
      <button onClick={handleSignup}>Sign Up</button>
      {message && <p>{message}</p>}
    </div>
  );
};

export default App;

Great! You've set up signup and login routes for the library system. However, users seem to have trouble logging in after they sign up. Examine the code and identify the issue causing login failures.

In [ ]:
//App.js (React frontend)
import { useState } from 'react';
import axios from 'axios';

export default function App() {
  const [password, setPassword] = useState('');
  const [userID, setUserID] = useState('');
  const [loginMessage, setLoginMessage] = useState(''); // New state variable for login message

  const handleSignup = () => {
    axios.post('/api/signup', { password }).then(response => {
      console.log(response.data);
      setUserID(response.data.userID);
    });
  };

  const handleLogin = () => {
    axios.post('/api/login', { userID, password })
      .then(response => setLoginMessage(response.data)) // Set login message on success
      .catch(err => setLoginMessage(err.message)); // Set error message on failure
  };

  return (
    <div>
      <input type="password" value={password} onChange={e => setPassword(e.target.value)} />
      <button onClick={handleSignup}>Sign Up</button>
      <button onClick={handleLogin}>Log In</button>
      {userID && <p>Your User ID: {userID}</p>}
      {loginMessage && <p>{loginMessage}</p>} {/* Display login message */}
    </div>
  );
}

In [ ]:
//index.js (Express server with Authentication Middleware)
const express = require('express');
const bcrypt = require('bcrypt');

const app = express();
app.use(express.json());
const users = {};

app.post('/api/signup', async (req, res) => {
  try {
    const hashedPassword = await bcrypt.hash(req.body.password, 10);
    const userID = 'user-' + Date.now();
    users[userID] = hashedPassword;  // req.body.password;
    res.status(200).send({ userID, hashedPassword });
  } catch (err) {
    res.status(500).send('Error during signup');
  }
});

app.post('/api/login', async (req, res) => {
  const userID = req.body.userID;
  const userHash = users[userID];

  try {
    const match = await bcrypt.compare(req.body.password, userHash);
    if (match) {
      res.status(200).send('Login Successful');
    } else {
      res.status(401).send('Login Failed');
    }
  } catch (err) {
    res.status(500).send('Error during login');
  }
});

app.listen(5000, () => console.log('Server running on port 5000'));

Alrighty, it's time to exercise your coding muscles again! In this practice task, I want you to secure the launch sequence of our Library System Management. We've added some safety nets by handling errors and securing passwords. However, there's still a piece missing in the server — the route handler that is responsible for hashing user passwords.

Your job is to create a hash of the user's password within the route handler where we handle the creation of new accounts or the authentication of existing users. This step is critical in ensuring that the application is secure and that user credentials are protected.

In [ ]:
//App.js (React frontend)
import { useState } from 'react';
import axios from 'axios';

export default function App() {
  const [signupPassword, setSignupPassword] = useState('');
  const [loginPassword, setLoginPassword] = useState('');
  const [signupStatus, setSignupStatus] = useState('');
  const [loginStatus, setLoginStatus] = useState('');
  
  const handleSignup = () => {
    axios.post('/api/signup', { password: signupPassword })
      .then(response => {
        setSignupStatus(response.data.message);
      })
      .catch(error => {
        setSignupStatus('Signup failed: ' + error.response.data.message);
      });
  };

  const handleLogin = () => {
    axios.post('/api/login', { password: loginPassword })
      .then(response => {
        setLoginStatus(response.data.message);
      })
      .catch(error => {
        setLoginStatus('Login failed: ' + error.message);
      });
  };

  return (
    <div>
      <div>
        <input
          type="password"
          value={signupPassword}
          onChange={(e) => setSignupPassword(e.target.value)}
          placeholder="Create Password"
        />
        <button onClick={handleSignup}>Sign Up</button>
        <p>{signupStatus}</p>
      </div>
      <div>
        <input
          type="password"
          value={loginPassword}
          onChange={(e) => setLoginPassword(e.target.value)}
          placeholder="Enter Password"
        />
        <button onClick={handleLogin}>Login</button>
        <p>{loginStatus}</p>
      </div>
    </div>
  );
}

In [ ]:
const express = require('express');
const bcrypt = require('bcrypt');
const app = express();
app.use(express.json());

const PORT = process.env.PORT || 5000;
const userDB = new Map();

app.post('/api/signup', async (req, res) => {
  const { password } = req.body;
  if (password.length < 6) {
    return res.status(400).json({ message: 'Password must be at least 6 characters' });
  }
  try {
    // TODO: Hash the password and store it securely using the bcrypt.hash method.
    // Update userDB to set a new username (generate it randomly or just hardcode for simplicity) and password.
    // Make sure that you handle any potential errors as well
    const hash = await bcrypt.hash(password, 10);
    userDB.set('user1', hash);
    
  } catch (error) {
    return res.status(500).json({ message: 'Error hashing the password' });
  }
});

// The rest of the code remains unchanged
app.post('/api/login', async (req, res) => {
  const { password } = req.body;
  try {
    const userHash = userDB.get('user1');
    const isMatch = userHash ? await bcrypt.compare(password, userHash) : false;
    if (isMatch) {
      return res.json({ message: 'Login successful' });
    } else {
      return res.status(401).json({ message: 'Password incorrect' });
    }
  } catch (error) {
    return res.status(500).json({ message: 'Error comparing the password' });
  }
});

app.listen(PORT, () => console.log(`Server running on port ${PORT}`));

Ready to create your own secure signup system using bcrypt? Your task is to set up a server capable of accepting a username and password, hashing the password, and storing the credentials. Ensure you validate the data and handle errors gracefully!

In [ ]:
//App.js (React frontend)
import { useState } from 'react';
import axios from 'axios';

export default function App() {
  const [username, setUsername] = useState('');
  const [password, setPassword] = useState('');
  const [message, setMessage] = useState('');

  const handleSignup = () => {
    axios.post('/api/signup', { username, password })
         .then(response => setMessage(response.data))
         .catch(error => setMessage(error.response.data));
  };

  const handleUsernameChange = (event) => {
    setUsername(event.target.value);
  };

  const handlePasswordChange = (event) => {
    setPassword(event.target.value);
  };

  return (
    <div>
      <h2>Signup</h2>
      <input type="text" value={username} onChange={handleUsernameChange} placeholder="Username" />
      <input type="password" value={password} onChange={handlePasswordChange} placeholder="Password" />
      <button onClick={handleSignup}>Signup</button>
      {message && <p>{message}</p>}
    </div>
  );
}

In [ ]:
// index.js (Express server) - before fix
const express = require('express');
const bcrypt = require('bcrypt');
const cors = require('cors');
const app = express();
const port = process.env.PORT || 5000;

app.use(cors());
app.use(express.json());

let users = {}; // This will store users

// TODO: Define a POST route for '/api/signup' to handle user signup.
// Within this route, take the provided username and password from the request,
// validate them – check if they're not empty
// If there is a user with such a username, return an error 409 with message 'Username is already taken!'
// otherwise hash the password and store the credentials in 'users' object.
// Remember to handle errors!

// TODO: Start the server and listen on a port

In [ ]:
// index.js (Express server) - francis complete